#### Classic RAG

In [1]:
!pip install psycopg2-binary SQLAlchemy sqlparse

In [2]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel

from llama_cpp import Llama
from huggingface_hub import hf_hub_download

import psycopg2 as pg
import sqlparse
engine = pg.connect("dbname='sales' user='zhigul' host='89.169.163.130' port='5432' password='asd'")


/home/zhigul/llm/venv_llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class BertEmbedder:
    def __init__(self, model_name, device='cpu'):
        """
        Инициализация BERT модели и токенайзера.
        
        :param model_name: Название предобученной модели).
        :param device: Устройство для вычислений ('cuda' для GPU или 'cpu' для CPU). Если None, автоматически выбирается доступное устройство.
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') if device is None else device
        print(device)
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        if self.device == 'cuda':
            self.model.cuda()

    def get_embedding(self, text):
        """
        Преобразует текст в эмбеддинг с использованием BERT.
        
        :param text: Входной текст для преобразования.
        :return: Эмбеддинг текста в виде numpy массива.
        """
        tokenized_text = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        
        with torch.no_grad():
            model_output = self.model(**{k: v.to(self.device) for k, v in tokenized_text.items()})
        
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings[0].cpu().numpy()
    
    def retrieve_reviews(self, query, reviews_df, top_n=3):
        review_texts = reviews_df['Review Text'].fillna('').tolist()
        
        reviews_embeddings = self.get_embedding(review_texts)
        query_embedding = self.get_embedding(query)
        similarities = cosine_similarity(query_embedding, reviews_embeddings).flatten()
        top_indices = similarities.argsort()[-top_n:][::-1]

        return reviews_df.iloc[top_indices]
    

In [4]:
class LLMModel:
    def __init__(self, model_id, filename_gguf, device='cpu'):
        """
        Инициализация LLM модели.
        
        :param model_name: Название предобученной модели.
        :param device: Устройство для вычислений ('cuda' для GPU или 'cpu' для CPU). Если None, автоматически выбирается доступное устройство.
        """
        # self.tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct')
        # model_path = hf_hub_download(repo_id=model_id, filename=filename_gguf)
        # self.model = Llama(
        #     model_path=model_path,
        #     n_gpu_layers=0,
        #     n_threads=16,
        #     n_ctx=4096
        # )
        self.generation_config = {
            "max_tokens": 20000,
            "stop": ["<|endoftext|>"],
            "echo": False,
            "n_ctx": 4096
        } 
        self.sql_llm = Llama.from_pretrained(
            repo_id="TheBloke/CodeLlama-7B-Instruct-GGUF",
            filename="codellama-7b-instruct.Q4_0.gguf",
            **self.generation_config
            )

        self.chat_llm = Llama.from_pretrained(
            repo_id="bartowski/llama3-turbcat-instruct-8b-GGUF",
            filename="llama3-turbcat-instruct-8b-Q4_K_S.gguf",
            **self.generation_config
            )


    # def generate_first_message(self):
    #     context_1 = """ Napoleon IT Отзывы - система для интеллектуального анализа и работы с пользовательским фидбэком. В том числе и на англоязычном рынке
    #     Как мы уже это сделали в Cotton Club и Lapochka.
    #     Sales & Operations
    #     -  Отслеживание качества обслуживания: Контроль качества обслуживания клиентов на всех этапах их взаимодействия, от оформления заказа до доставки
    #     -  Оптимизация ассортимента: Выявление наиболее популярных и востребованных позиций, а также тех, которые требуют улучшения
    #     Brand Management & Consumer Market Insights (CMI) 
    #     -  Отслеживание реакции на кампании: Мониторинг изменения восприятия до, во время и после рекламных кампаний
    #     RnD
    #     -  Тестирование нововведений: Анализ отзывов о новых продуктах и услугах, чтобы понять их восприятие и эффективность
    #     -  Анализ трендов: Отслеживание трендов в индустрии, чтобы своевременно внедрять актуальные изменения и оставаться конкурентоспособными
    #     """
    #     system_prompt = f"""Ты профессиональный агент-продавец, опыт которого в продажах больше 10 лет.
    #         Ты инициализируешь диалог с клиентом и твоя цель - заинтересовать его продуктом, который ты предлагаешь.
    #         Ты должен быть убедительным и дружелюбным. Обращайся на 'Вы', но не используй в письме вступление в виде 'Уважаемый ...' и заключение по типу 'С уважением..'
    #         Твои ответы должны быть только на Русском.

    #         Продукт, который ты предлагаешь описан тут:
    #         {context_1}

    #         Покажи уверенные навыки продавца и заинтересуй клиента:
    #         """

    #     user_prompt = f"Я - потенциальный клиент. Инициируй со мной диалог: Напиши мне письмо, где рассказываешь о продукте Napoleon Отзывы."

    #     message = [
    #         {'role': 'system', 'content': system_prompt},
    #         {'role': 'user', 'content': user_prompt}
    #     ]
        
    #     # prompt = self.tokenizer.apply_chat_template(message, tokenize=False).rstrip('<|endoftext|>').strip() + '\n<|assistant|>' # tokenize=False => в prompt будет храниться строка со спец.токенами
    #     # llm = self.model(prompt, **self.generation_config)
    #     result = self.llm.create_chat_completion(messages = message)

    #     print(f"Ответ модели: {result['choices'][0]['message']['content']}")

    def classify_rag_or_text2sql(self, client_query):
        system_prompt = f"""Ты должен будешь определить, можно-ли перевести отправленое тебе предложение в SQL. 
                            Ты должен отвечать на вопросы только 'Да' и 'Нет'.
                        """
        user_prompt = client_query

        message = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]
        result = self.chat_llm.create_chat_completion(messages=message)
        print(f"Ответ модели: {result['choices'][0]['message']['content']}")
        return result['choices'][0]['message']['content']

    def generate_text2sql_response(self, client_query):
        schema = """CREATE TABLE reviews (
        id INTEGER DEFAULT nextval('reviews_id_seq'::regclass) NOT NULL, 
        company_id INTEGER NOT NULL, 
        product_cat VARCHAR, 
        product_name VARCHAR, 
        review_dt TIMESTAMP, 
        review_text VARCHAR, 
        topic VARCHAR, 
        sentiment VARCHAR, 
        marketplace VARCHAR, 
        embedding VECTOR(20), 
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 
        deleted_at TIMESTAMP, 
        CONSTRAINT reviews_pkey PRIMARY KEY (id), 
        CONSTRAINT reviews_company_id_fkey FOREIGN KEY(company_id) REFERENCES companies (id)
            )      
        """
        sql_example = "SELECT * FROM reviews WHERE topic == 'Вкус"
        user_prompt = client_query
        system_prompt = f"""
                Generate a Postgresql SQL query to answer the following question:
                `{user_prompt}
                Please wrap your code answer using ```sql:
                ### Database Schema
                This query will run on a database whose schema is represented in this string:
                Don't use joins for this schema and if all columns are required give the (*) notation.
                `{schema}`
                An example of the SQL would be `{sql_example}`
                ### SQL
                Given the database schema, here is the SQL query that answers `{user_prompt}`:
                ```sql
                """

        message = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]
        result = self.sql_llm.create_chat_completion(messages=message)
        print(f"Ответ модели: {sqlparse.format(result['choices'][0]['message']['content'].split("```")[-2], reindent=True).replace('#','').replace("sql ","")}")

    def generate_rag_response(self, client_query, relevant_reviews):
        review_texts = '\n'.join(relevant_reviews['Review Text'].tolist())
        system_prompt = f"""Ты профессиональный агент-продавец, опыт которого в продажах больше 10 лет.
            Ты ведешь диалог с клиентом, которому предлагаешь сотрудничество по продукту "Napoleon IT Отзывы".
            Ты должен обрабатывать любые вопросы потенциального клиента, предоставляя только релевантную и достовернную информацию, используя контекст.
            Не груби и будь дружелюбным собесебником.
            Отвечай только на Русском.

            Обработай запрос клиента и выдай точную информацию:
            Вот отзывы о товарах клиента: {review_texts}. Ответь на вопрос клиента на опираясь на них.
            """
        user_prompt = f"Клиент спросил: '{client_query}'"

        message = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]
        
        result = self.chat_llm.create_chat_completion(messages=message)
        
        print(f"Ответ модели: {result['choices'][0]['text']}")

In [5]:
class SalesBotHandler:
    def __init__(self,
                 bert_model_name='cointegrated/rubert-tiny', 
                 llm_model_name='TheBloke/CodeLlama-7B-Instruct-GGUF',
                 llm_model_path='codellama-7b-instruct.Q4_0.gguf',
                 reviews_path = "/home/zhigul/llm/cleaned_coffee.csv"):
        self.bert_embedder = BertEmbedder(model_name=bert_model_name)
        self.llm_model = LLMModel(model_id=llm_model_name, filename_gguf=llm_model_path)
        self.reviews = self.load_reviews(reviews_path)
    
    def load_reviews(self, reviews_path):
        return pd.read_csv(reviews_path)
    
    def process_customer_input(self, customer_input):
        """
        Метод для ведения диалога с клиентом с использованием LLM Model.
        """
        # relevant_reviews = self.bert_embedder.retrieve_reviews(customer_input, self.reviews)
        is_text2sql = self.llm_model.classify_rag_or_text2sql(customer_input)
        if 'Да' in is_text2sql:
            print('ЭТО TEXT2SQL')
            return self.llm_model.generate_text2sql_response(customer_input)
        else:
            print('ЭТО ПРОСТО RAG')
            # return self.llm_model.generate_rag_response(customer_input, relevant_reviews)

    def generate_first_message(self):
        """
        Метод для генерации первого письма с использованием LLM Model.
        """
        return self.llm_model.generate_first_message()

In [6]:
handler = SalesBotHandler()

cpu


/home/zhigul/llm/venv_llm/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /home/zhigul/.cache/huggingface/hub/models--TheBloke--CodeLlama-7B-Instruct-GGUF/snapshots/2f064ee0c6ae3f025ec4e392c6ba5dd049c77969/./codellama-7b-instruct.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = codellama_codellama-7b-instruct-hf
llama_

In [7]:
# handler.generate_first_message()

client_query = "Сколько положительных отзывов о моих товарах?"
relevant_reviews = handler.process_customer_input(client_query)


llama_print_timings:        load time =    5661.95 ms
llama_print_timings:      sample time =       0.40 ms /     3 runs   (    0.13 ms per token,  7481.30 tokens per second)
llama_print_timings: prompt eval time =    5661.77 ms /    69 tokens (   82.05 ms per token,    12.19 tokens per second)
llama_print_timings:        eval time =     300.88 ms /     2 runs   (  150.44 ms per token,     6.65 tokens per second)
llama_print_timings:       total time =    5967.92 ms /    71 tokens


Ответ модели: Да
ЭТО TEXT2SQL



llama_print_timings:        load time =   54194.34 ms
llama_print_timings:      sample time =       5.14 ms /   124 runs   (    0.04 ms per token, 24110.44 tokens per second)
llama_print_timings: prompt eval time =   54193.86 ms /   405 tokens (  133.81 ms per token,     7.47 tokens per second)
llama_print_timings:        eval time =   27376.00 ms /   123 runs   (  222.57 ms per token,     4.49 tokens per second)
llama_print_timings:       total time =   81659.31 ms /   528 tokens


Ответ модели: sql
SELECT COUNT(*)
FROM reviews
WHERE sentiment = 'positive'
  AND company_id = <your_company_id>;
